In [1]:
import numpy as np
import pandas as pd
import string
import re
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import product
import warnings

warnings.filterwarnings('ignore')

In [2]:
punc = string.punctuation
num = string.digits

In [3]:
jobs = pd.read_csv('df_full.csv')

In [4]:
jobs.head()

,Title,Company,Location,Pay,Description
0,['Reception/ Administration/High volume Invoic...,['BT Recruitment Pty Ltd'],['Melbourne'],[],"['<div data-automation=""mobileTemplate"" class=..."
1,['Lead Data Architect'],['ME'],['Melbourne'],[],"['<div data-automation=""mobileTemplate"" class=..."
2,['Data Analyst'],['Victoria University'],['Melbourne'],[],"['<div data-automation=""mobileTemplate"" class=..."
3,['Data Wrangling Expert/Asset Performance Anal...,['Peak Services'],['Melbourne'],[],"['<div data-automation=""mobileTemplate"" class=..."
4,['Data Scientist / Data Analyst'],['Randstad - Technologies'],['Melbourne'],[],"['<div data-automation=""mobileTemplate"" class=..."


In [5]:
jobs = jobs.drop_duplicates(subset=['Title', 'Location', 'Description'], keep=False)

In [6]:
jobs.reset_index(inplace=True)

In [7]:
jobs.drop(columns='index', axis=1, inplace=True)

In [8]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797 entries, 0 to 796
Data columns (total 5 columns):
Title          797 non-null object
Company        797 non-null object
Location       797 non-null object
Pay            797 non-null object
Description    797 non-null object
dtypes: object(5)
memory usage: 31.2+ KB


In [9]:
jobs['Title'] = jobs['Title'].str.strip(punc).str.lower()

In [10]:
jobs['Company'] = jobs['Company'].str.strip(punc).str.lower()

In [11]:
jobs['Location'] = jobs['Location'].str.strip(punc).str.lower()

In [12]:
jobs['Pay'] = jobs['Pay'].str.strip('[]').str.strip("''")

In [13]:
jobs['Description'] = jobs['Description'].str.strip(punc).str.strip(num).str.lower()

In [14]:
jobs.Pay = jobs.Pay.replace('', np.NaN)

In [15]:
jobs.head()

,Title,Company,Location,Pay,Description
0,lead data architect,me,melbourne,NaN,"div data-automation=""mobiletemplate"" class=""_2..."
1,data wrangling expert/asset performance analyst,peak services,melbourne,NaN,"div data-automation=""mobiletemplate"" class=""_2..."
2,data scientist / data analyst,randstad - technologies,melbourne,NaN,"div data-automation=""mobiletemplate"" class=""_2..."
3,data analyst,nab,melbourne,NaN,"div data-automation=""mobiletemplate"" class=""_2..."
4,analyst chapter lead (cloud and big data,anz,melbourne,NaN,"div data-automation=""mobiletemplate"" class=""_2..."


In [16]:
def get_salary_hour(x):
    if x is np.nan:
        return False
    elif re.search("hour|ph",x.replace('.', '')):
        return True
    else:
        return False

In [17]:
def clean_salary_hour(x):
    try :
        a = float(re.findall("(\d+[\,|\ ]+\d+)|(\d+)", x)[0][1].replace('.', ''))
        b = float(re.findall("(\d+[\,|\ ]+\d+)|(\d+)", x)[1][1].replace('.', ''))
        c = (a + b) / 2
        return c * 2080
    
    
    except :
        a = float(re.findall("(\d+[\,|\ ]+\d+)|(\d+)", x)[0][1].replace('.', ''))
        return a * 2080

In [18]:
jobs['hour'] = jobs.Pay[jobs.Pay.apply(get_salary_hour)].apply(clean_salary_hour)

In [19]:
def get_salary_day(x):
    if x is np.nan:
        return False
    elif re.search(".*\d+.*(day)|(pd[^\w])",x.replace('.', '')):
        return True
    else:
        return False

In [20]:
def clean_salary_day(x):
    try :
        results = re.findall("(\d+[\,|\ |\.]+\d+|\d+)", x.replace(',',''))
        
        a = results[0]
        b = results[1]

        c = (float(a) + float(b) / 2)

        return c * 260

    except :
        results = re.findall("(\d+[\,|\ |\.]+\d+|\d+)", x.replace(',',''))

        return float(results[0]) * 260

In [21]:
jobs['day'] = jobs.Pay[jobs.Pay.apply(get_salary_day)].map(clean_salary_day)

In [22]:
def get_salary_year(x):
    if x is np.nan:
        return False
    elif re.search("\d+(year)|(pa[^\w])|(annum)",x.replace('.', '')):
        return True
    else:
        return False

In [23]:
def clean_salary_year(x):
    try :
        results = re.findall("(\d+[\,|\ |\.]+\d+|\d+)", x.replace(',',''))

        a = results[0]
        b = results[1]

        c = (float(a) + float(b) / 2)

        if c < 1000:
            c = c * 1000
        return c

    except :
        results = re.findall("(\d+[\,|\ |\.]+\d+|\d+)", x.replace(',',''))

        return float(results[0])

In [24]:
jobs.Pay.loc[749] = '$100k - $130k p.a. + bonus'

In [25]:
jobs.Pay.loc[1133] = '$600 - $700 p.d. + including super'

In [26]:
jobs['year'] = jobs.Pay[jobs.Pay.apply(get_salary_year)].map(clean_salary_year)

In [27]:
jobs['year'].fillna(0, inplace=True)

In [28]:
jobs['day'].fillna(0, inplace=True)

In [29]:
jobs['hour'].fillna(0, inplace=True)

In [30]:
jobs['total_pay'] = jobs.year + jobs.day + jobs.hour

In [31]:
jobs.head()

,Title,Company,Location,Pay,Description,hour,day,year,total_pay
0,lead data architect,me,melbourne,NaN,"div data-automation=""mobiletemplate"" class=""_2...",0.0,0.0,0.0,0.0
1,data wrangling expert/asset performance analyst,peak services,melbourne,NaN,"div data-automation=""mobiletemplate"" class=""_2...",0.0,0.0,0.0,0.0
2,data scientist / data analyst,randstad - technologies,melbourne,NaN,"div data-automation=""mobiletemplate"" class=""_2...",0.0,0.0,0.0,0.0
3,data analyst,nab,melbourne,NaN,"div data-automation=""mobiletemplate"" class=""_2...",0.0,0.0,0.0,0.0
4,analyst chapter lead (cloud and big data,anz,melbourne,NaN,"div data-automation=""mobiletemplate"" class=""_2...",0.0,0.0,0.0,0.0


In [32]:
simple = []
for i in jobs.Title:
    if 'analyst' in i:
        simple.append("Analyst")
    elif 'engineer' in i:
        simple.append("Engineer")
    elif 'scientist' in i:
        simple.append("Scientist")
    elif 'consultant' in i:
        simple.append("Consultant")
    else:
        simple.append("Other")

In [33]:
jobs['Simple_Title'] = simple

In [34]:
jobs.head()

,Title,Company,Location,Pay,Description,hour,day,year,total_pay,Simple_Title
0,lead data architect,me,melbourne,NaN,"div data-automation=""mobiletemplate"" class=""_2...",0.0,0.0,0.0,0.0,Other
1,data wrangling expert/asset performance analyst,peak services,melbourne,NaN,"div data-automation=""mobiletemplate"" class=""_2...",0.0,0.0,0.0,0.0,Analyst
2,data scientist / data analyst,randstad - technologies,melbourne,NaN,"div data-automation=""mobiletemplate"" class=""_2...",0.0,0.0,0.0,0.0,Analyst
3,data analyst,nab,melbourne,NaN,"div data-automation=""mobiletemplate"" class=""_2...",0.0,0.0,0.0,0.0,Analyst
4,analyst chapter lead (cloud and big data,anz,melbourne,NaN,"div data-automation=""mobiletemplate"" class=""_2...",0.0,0.0,0.0,0.0,Analyst


In [35]:
jobs.Simple_Title.value_counts()

Analyst       417
Other         246
Engineer       56
Scientist      43
Consultant     35
Name: Simple_Title, dtype: int64

In [36]:
level = []
for i in jobs.Title:
    if 'senior' in i:
        level.append("Senior")
    elif 'junior' in i:
        level.append("Junior")
    else:
        level.append("Middle")

In [37]:
jobs['Job_Level'] = level

In [38]:
jobs.Job_Level.value_counts()

Middle    670
Senior    112
Junior     15
Name: Job_Level, dtype: int64

In [39]:
no_salary = jobs[jobs.total_pay == 0]

In [40]:
salary = jobs[jobs.total_pay != 0]

In [41]:
print(len(salary) + len(no_salary))

797


In [42]:
for a, b in product(jobs['Simple_Title'].unique(), jobs['Job_Level'].unique()):
    missing1, missing2 = no_salary['Simple_Title']==a, salary['Simple_Title']==a
    missing3, missing4 = no_salary['Job_Level']==b, salary['Job_Level']==b
    no_salary.loc[missing1 & missing3, 'total_pay'] = salary.loc[missing2 & missing4, 'total_pay'].median()

In [43]:
jobs = pd.concat([salary, no_salary], )

In [44]:
jobs.sort_index(inplace=True)

In [45]:
jobs.drop(columns=['Pay', 'hour', 'day', 'year'], axis=1, inplace=True)

In [46]:
jobs.head()

,Title,Company,Location,Description,total_pay,Simple_Title,Job_Level
0,lead data architect,me,melbourne,"div data-automation=""mobiletemplate"" class=""_2...",159750.0,Other,Middle
1,data wrangling expert/asset performance analyst,peak services,melbourne,"div data-automation=""mobiletemplate"" class=""_2...",201500.0,Analyst,Middle
2,data scientist / data analyst,randstad - technologies,melbourne,"div data-automation=""mobiletemplate"" class=""_2...",201500.0,Analyst,Middle
3,data analyst,nab,melbourne,"div data-automation=""mobiletemplate"" class=""_2...",201500.0,Analyst,Middle
4,analyst chapter lead (cloud and big data,anz,melbourne,"div data-automation=""mobiletemplate"" class=""_2...",201500.0,Analyst,Middle


In [47]:
jobs.dropna(inplace=True)

In [48]:
jobs.shape

(769, 7)

In [49]:
jobs.to_csv('jobs.csv', index=False)

In [51]:
print(jobs.total_pay.unique())

[159750.   201500.   221000.   312000.   101750.   250250.    27040.
  28080.   149500.   197667.25 187500.   215000.   185000.   130000.
 180000.   170000.    72800.   253500.   182000.   325000.   145000.
 351000.   247000.   124800.   117500.   195000.    97760.    88400.
 190000.   182130.   156000.   148550.   208000.   290000.   140000.
 260000.    93600.   123500.   214500.   760000.   125000.   227500.
  80000.   357500.   286000.   273000.    65520.   144560.   390000.
 266500.   204100.   280000.   220000.   167574.5  226200.   165000.
 227760.    95060.   950000.  ]


In [66]:
total = []
for x in jobs.total_pay:
    if x > 400000:
        x = x / 10
        total.append(x)
       
    else:
        total.append(x)

In [67]:
jobs.total_pay = total

In [74]:
print(jobs.total_pay.unique())

[159750.   201500.   221000.   312000.   101750.   250250.    27040.
  28080.   149500.   197667.25 187500.   215000.   185000.   130000.
 180000.   170000.    72800.   253500.   182000.   325000.   145000.
 351000.   247000.   124800.   117500.   195000.    97760.    88400.
 190000.   182130.   156000.   148550.   208000.   290000.   140000.
 260000.    93600.   123500.   214500.    76000.   125000.   227500.
  80000.   357500.   286000.   273000.    65520.   144560.   390000.
 266500.   204100.   280000.   220000.   167574.5  226200.   165000.
 227760.    95060.    95000.  ]


In [75]:
jobs.to_csv('job_viz.csv', index=False)